# ** Implementation of multi-layer neural network using Tensorflow**

**Import libraries**

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

### **Version with explicit layers definition**

In [44]:
batch_size = 128
no_of_batches = 10000
tf.reset_default_graph() # restart model graph

# placeholder for inputs
inputs = tf.placeholder(tf.float32, [None, 2])
labels = tf.placeholder(tf.float32, [None, 1])

val_inputs = tf.placeholder(tf.float32, [None, 2])
val_labels = tf.placeholder(tf.float32, [None, 1])

def define_variable(x):
    return tf.Variable(tf.random.truncated_normal(x, mean=0.0,stddev=0.1))

#define layers
w_1 = define_variable([2,128])
b_1 = define_variable([1,128])

w_2 = define_variable([128,64])
b_2 = define_variable([1,64])

w_3 = define_variable([64,1])
b_3 = define_variable([1,1])

#training forward pass
logits_1 = tf.matmul(inputs, w_1) + b_1
acivations_1 = tf.nn.relu(logits_1)

logits_2 = tf.matmul(acivations_1, w_2) + b_2
acivations_2 = tf.nn.relu(logits_2)

logits_3 = tf.matmul(acivations_2, w_3) + b_3

error = tf.losses.mean_squared_error(logits_3, labels)
step = tf.train.GradientDescentOptimizer(0.05).minimize(error)

##validation forward pass
logits_1 = tf.matmul(val_inputs, w_1) + b_1
acivations_1 = tf.nn.relu(logits_1)

logits_2 = tf.matmul(acivations_1, w_2) + b_2
acivations_2 = tf.nn.relu(logits_2)

logits_3 = tf.matmul(acivations_2, w_3) + b_3

val_error = tf.losses.mean_squared_error(logits_3, val_labels)

**Initialise session**

In [45]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run([init])

[None]

**Create dataset**

In [46]:
np.random.seed(3)
features = 0.2 * np.random.randn(batch_size * no_of_batches, 2) + 0.9
label = features[:,0] * features[:,1]**3

**Train network**

In [47]:
for _ in range(5):
    for i in range(no_of_batches-1):
        feed = {inputs: features.reshape(-1, batch_size, 2)[i], labels:label.reshape(-1, batch_size, 1)[i].reshape((-1,1)),
           val_inputs: features.reshape(-1, batch_size, 2)[no_of_batches-1], val_labels:label.reshape(-1, batch_size, 1)[no_of_batches-1].reshape((-1,1))} 

        _, error_out,  val_error_out = sess.run([step, error, val_error], feed_dict=feed)
    print ("train error: {} validation error: {}".format(error_out, val_error_out))
    

train error: 3.643821764853783e-05 validation error: 0.00013497892359737307
train error: 3.989394463133067e-05 validation error: 7.715729589108378e-05
train error: 3.23363856296055e-05 validation error: 5.09434103150852e-05
train error: 3.1486233638133854e-05 validation error: 4.5261029299581423e-05
train error: 2.979454438900575e-05 validation error: 4.090691436431371e-05


### **Version with parametrised layers**

In [48]:
batch_size = 128
no_of_batches = 10000
tf.reset_default_graph() # restart model graph

# placeholder for inputs
inputs = tf.placeholder(tf.float32, [None, 2])
labels = tf.placeholder(tf.float32, [None, 1])

def define_variable(x):
    return tf.Variable(tf.random.truncated_normal(x, mean=0.0,stddev=0.1))

layers = [128,128, 64, 64]; weights_list_dims = [2] + layers + [1]; biases_list_dims = layers + [1]
weights_list = [define_variable([weights_list_dims[x],weights_list_dims[x+1]])  for x in range(len(weights_list_dims)-1 )]
biases_list = [define_variable([1,biases_list_dims[x]])  for x in range(len(biases_list_dims) )]

activations = tf.identity(inputs)

#forward pass of the data
for i in range(len(layers)):#
    logits = tf.matmul(activations, weights_list[i]) + biases_list[i]
    activations = tf.nn.relu(logits)

logits = tf.matmul(activations, weights_list[-1]) + biases_list[-1]

error = tf.losses.mean_squared_error(logits, labels)
step = tf.train.GradientDescentOptimizer(0.05).minimize(error)


**Initialise session**

In [49]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run([init])

[None]

**Create dataset**

In [50]:
np.random.seed(3)
features = 0.2 * np.random.randn(batch_size * no_of_batches, 2) + 0.9
label = features[:,0] * features[:,1]**3

**Train network**

In [51]:
for _ in range(5):
    for i in range(no_of_batches-1):
        #train
        feed = {inputs: features.reshape(-1, batch_size, 2)[i], labels:label.reshape(-1, batch_size, 1)[i].reshape((-1,1))}
        _, error_out = sess.run([step, error], feed_dict=feed)
        #validation
        feed = {inputs: features.reshape(-1, batch_size, 2)[no_of_batches-1], labels:label.reshape(-1, batch_size, 1)[no_of_batches-1].reshape((-1,1))} 
        val_error_out, _ = sess.run([error, logits], feed_dict=feed)
    print ("train error: {} validation error: {}".format(error_out, val_error_out))
    

train error: 0.00013864244101569057 validation error: 0.000266754679614678
train error: 4.128673754166812e-05 validation error: 9.55739087658003e-05
train error: 2.6525107386987656e-05 validation error: 5.914747089263983e-05
train error: 2.29969373322092e-05 validation error: 4.485086537897587e-05
train error: 1.541944220662117e-05 validation error: 3.2426109100924805e-05
